In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Some libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import seaborn as sns

input_file = np.load('/content/drive/My Drive/public_data.npz', allow_pickle=True)
data = input_file['data']

labels = input_file['labels']

label_dict = {'healthy': 0, 'unhealthy': 1}
labels = np.array([label_dict[label] for label in labels])


2.14.0


In [ ]:
#data = data/255.0

from keras.applications.efficientnet_v2 import preprocess_input
data = preprocess_input(data)

In [ ]:
shrek_indices = []
trol_indices = []
new_data = []
new_labels = []
for i, image in enumerate(data):
  if np.sum(data[506] - image) == 0:
    shrek_indices.append(i)
  elif np.sum(data[338] - image) == 0:
    trol_indices.append(i)
  else:
    new_data.append(image)
    new_labels.append(labels[i])

images = np.array(new_data)
labels = np.array(new_labels)
labels = tfk.utils.to_categorical(labels)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=seed)

In [ ]:
input_shape = X_train.shape[1:]  # Input shape for the model

In [ ]:
learned_model = tfk.applications.EfficientNetV2M(
    input_shape=(96, 96, 3),
    include_top=False,
    weights="imagenet",
    pooling='max',
)

In [ ]:
learned_model.trainable = True

In [ ]:
N = 200
for i, layer in enumerate(learned_model.layers[:N]):
  layer.trainable=False

In [ ]:
preprocessing = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal_and_vertical"),
    tfkl.RandomRotation(0.5),
    #tfkl.GaussianNoise(0.3),
    #tfkl.RandomContrast(0.4),
    #tfkl.RandomBrightness(0.4),
], name='preprocessing')


input_layer = tfkl.Input(shape=input_shape)
x = preprocessing(input_layer)

In [ ]:
x = learned_model(x)

In [ ]:
from keras import regularizers

x = tfkl.Flatten(name='flattenLast')(x)
x = tfkl.Dropout(0.1)(x)
x = tfkl.Dense(512, activation='relu', name='Dense_1')(x)
x = tfkl.Dropout(0.1)(x)
x = tfkl.Dense(256, activation='relu', name='Dense_2')(x)
x = tfkl.Dropout(0.1)(x)
x = tfkl.Dense(128, activation='relu', name='Dense_3')(x)
x = tfkl.Dropout(0.1)(x)
x = tfkl.Dense(64, activation='relu', name='Dense_4')(x)
x = tfkl.Dense(16, activation='relu', name='Dense_5')(x)
output_layer = tfkl.Dense(2, activation='sigmoid', name='output')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.AdamW(1e-5), metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 preprocessing (Sequential)  (None, 96, 96, 3)         0         
                                                                 
 efficientnetv2-m (Function  (None, 1280)              53150388  
 al)                                                             
                                                                 
 flattenLast (Flatten)       (None, 1280)              0         
                                                                 
 dropout_4 (Dropout)         (None, 1280)              0         
                                                                 
 Dense_1 (Dense)             (None, 512)               655872    
                                                           

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

In [ ]:
model_checkpoint = tfk.callbacks.ModelCheckpoint('best_model_finetune.h5', monitor='val_accuracy', save_best_only=True, mode='max')

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
#class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
#class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {0: 0.81, 1: 1.29}

print(f"Class weights: {class_weights_dict}")

Class weights: {0: 0.81, 1: 1.29}


In [ ]:
batch_size=16
epochs=34#100
history = model.fit(
    x=images,
    y=labels,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    class_weight=class_weights_dict,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

Epoch 1/34
313/313 [==============================] - 168s 225ms/step - loss: 0.6979 - accuracy: 0.5791 - val_loss: 0.6659 - val_accuracy: 0.6503
Epoch 2/34
313/313 [==============================] - 63s 200ms/step - loss: 0.6518 - accuracy: 0.6631 - val_loss: 0.5578 - val_accuracy: 0.7473
Epoch 3/34
313/313 [==============================] - 63s 203ms/step - loss: 0.5669 - accuracy: 0.7402 - val_loss: 0.4426 - val_accuracy: 0.8022
Epoch 4/34
313/313 [==============================] - 64s 206ms/step - loss: 0.4896 - accuracy: 0.7662 - val_loss: 0.3928 - val_accuracy: 0.8232
Epoch 5/34
313/313 [==============================] - 61s 196ms/step - loss: 0.4627 - accuracy: 0.7832 - val_loss: 0.3587 - val_accuracy: 0.8511
Epoch 6/34
313/313 [==============================] - 71s 228ms/step - loss: 0.4262 - accuracy: 0.7966 - val_loss: 0.3357 - val_accuracy: 0.8531
Epoch 7/34
313/313 [==============================] - 69s 219ms/step - loss: 0.4247 - accuracy: 0.8090 - val_loss: 0.3216 - val_a

In [ ]:
# Evaluate the model on the test set
#model.load_weights("best_model.h5")
test_loss, test_accuracy = model.evaluate(X_val, y_val, verbose=1)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

32/32 [==============================] - 2s 71ms/step - loss: 0.1300 - accuracy: 0.9600
Test loss: 0.12996630370616913
Test accuracy: 0.9600399732589722


In [ ]:
model.save('saved_model_LASTLASTLAST')